# TODO:
## Baratin:
- dire qu'on gere les nouveaux users en leur demandant les feat/category qu'ils kiffent
## User features:
- avg watch ratio in each video feat_*
- avg watch ratio for each first level category id of the video captions
## Video features:
- vector encoding of the feat_*
- vector encoding of the first level category id of the caption
- its length
- engagement score (groups likes, comments, downloads, etc.) sur les 7 derniers jours (trend)

# Imports

In [1]:
import pandas as pd
import os
import numpy as np

# Loading the data

In [2]:
export_dir = "./exports"
big_matrix_cleaned = pd.read_parquet(export_dir + "/big_matrix_cleaned.pq")
#small_matrix_cleaned = pd.read_parquet(export_dir + "/small_matrix_cleaned.pq")
item_categories_cleaned = pd.read_parquet(export_dir + "/item_categories_cleaned.pq")
item_daily_features_cleaned = pd.read_parquet(export_dir + "/item_daily_features_cleaned.pq")
caption_category_cleaned = pd.read_parquet(export_dir + "/caption_category_cleaned.pq")

# CREATING USER VECTORS

In [3]:
user_avg_feat = big_matrix_cleaned.copy()
user_avg_feat = big_matrix_cleaned[["user_id", "video_id", "watch_ratio"]].copy()

user_avg_feat = user_avg_feat.merge(item_categories_cleaned, on="video_id", how="left")
user_avg_feat = user_avg_feat.explode(column="feat")
user_avg_feat['avg_feat'] = user_avg_feat.groupby(['user_id', 'feat'])['watch_ratio'].transform('mean').fillna(0)

white_list = ["user_id", "feat", "avg_feat"]

user_avg_feat = user_avg_feat[white_list].drop_duplicates()

user_avg_feat = user_avg_feat.sort_values(["user_id", "feat"]).reset_index(drop=True)

user_avg_feat = user_avg_feat.pivot(
    index='user_id',
    columns='feat',
    values='avg_feat'
)

user_avg_feat.columns = [f'avg_feat_{int(col)}' for col in user_avg_feat.columns]

# Replace NaNs with 0
user_avg_feat = user_avg_feat.fillna(0)

# Reset index if you want 'user_id' back as a column
user_avg_feat = user_avg_feat.reset_index()
user_avg_feat

,user_id,avg_feat_0,avg_feat_1,avg_feat_2,avg_feat_3,avg_feat_4,avg_feat_5,avg_feat_6,avg_feat_7,avg_feat_8,...,avg_feat_21,avg_feat_22,avg_feat_23,avg_feat_24,avg_feat_25,avg_feat_26,avg_feat_27,avg_feat_28,avg_feat_29,avg_feat_30
0,0,1.279674,0.977637,0.909355,0.902536,0.762750,1.156619,0.857332,1.116271,1.157333,...,1.092595,0.229430,1.172116,2.434744,1.024753,1.157281,0.844790,1.145681,1.043639,0.485446
1,1,0.365590,0.747119,0.285845,0.183951,1.644048,0.932794,0.908008,1.039110,0.993866,...,1.210433,0.000000,0.102452,0.000000,0.774030,1.184769,2.134150,1.123180,1.434024,1.488989
2,2,0.376160,0.728525,0.481671,0.558891,0.910193,0.685176,0.513376,0.658626,0.668409,...,1.013190,0.000000,0.771147,0.000000,0.611067,0.725683,0.210522,0.661212,0.176768,0.079384
3,3,1.450146,0.950613,0.775522,0.796545,1.047035,1.208812,0.824149,1.009434,1.053580,...,0.897935,0.166655,0.529580,0.000000,0.956464,0.887288,0.623670,1.036636,1.561361,0.543257
4,4,0.000000,0.476605,1.010854,0.000000,0.455023,0.330529,0.491453,0.716446,0.744431,...,0.800958,0.000000,0.000000,0.000000,1.021075,0.966819,0.000000,0.922474,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6894,7171,0.603449,0.899103,0.717087,0.526626,0.739019,0.910404,0.662487,0.884767,0.764045,...,0.279137,0.019471,0.416539,0.000000,0.865869,0.829535,0.846472,0.994376,0.181914,0.681021
6895,7172,0.819361,1.042957,0.793120,0.496253,0.647006,0.962898,0.987411,1.039611,1.269542,...,0.989900,0.058764,1.305065,0.000000,0.963322,1.183976,1.273557,1.183512,0.000000,0.522903
6896,7173,0.293560,0.273140,1.719037,0.000000,0.375374,0.478506,0.726166,0.684708,0.609157,...,0.000000,0.000000,0.441495,0.000000,0.839305,0.771188,0.000000,0.864359,0.000000,0.585139
6897,7174,1.071403,0.710077,0.527994,0.851187,0.932450,0.849032,0.673943,0.852725,0.865874,...,0.703509,0.083650,3.462550,0.000000,0.821990,0.786393,0.765432,0.960470,0.831416,0.772563


In [4]:
user_avg_category = big_matrix_cleaned.copy()
user_avg_category = user_avg_category.merge(caption_category_cleaned, on="video_id", how="left")
user_avg_category = user_avg_category.explode(column="first_level_category_id")
user_avg_category['avg_category'] = user_avg_category.groupby(['user_id', 'first_level_category_id'])['watch_ratio'].transform('mean').fillna(0)

white_list = ["user_id", "first_level_category_id", "avg_category"]

user_avg_category = user_avg_category[white_list].drop_duplicates()

user_avg_category = user_avg_category.sort_values(["user_id", "first_level_category_id"]).reset_index(drop=True)

user_avg_category = user_avg_category.pivot(
    index='user_id',
    columns='first_level_category_id',
    values='avg_category'
)

user_avg_category.columns = [f'avg_category_{int(col)}' for col in user_avg_category.columns]

# Replace NaNs with 0
user_avg_category = user_avg_category.fillna(0)

# Reset index if you want 'user_id' back as a column
user_avg_category = user_avg_category.reset_index()
user_avg_category

,user_id,avg_category_1,avg_category_2,avg_category_3,avg_category_4,avg_category_5,avg_category_6,avg_category_7,avg_category_8,avg_category_9,...,avg_category_30,avg_category_31,avg_category_32,avg_category_33,avg_category_34,avg_category_35,avg_category_36,avg_category_37,avg_category_38,avg_category_39
0,0,1.068750,0.884540,0.949826,0.921405,1.136661,0.922503,1.131475,1.249193,1.091720,...,1.410021,1.178845,0.787215,1.219192,0.985790,1.023252,0.876753,0.836855,0.586343,1.098977
1,1,0.807564,0.324609,0.053281,0.993688,1.064761,1.041215,1.074544,1.071133,0.937302,...,1.234668,1.083152,0.605532,0.635479,0.706868,1.121345,0.992612,1.489968,3.278239,0.253628
2,2,0.744794,0.345140,0.558891,0.909273,0.691760,0.739852,0.704704,0.768956,0.621410,...,0.885171,0.539263,0.461030,0.678805,0.598511,0.581113,0.464605,0.446583,0.000000,0.000000
3,3,0.942274,0.756333,0.788544,1.095945,1.146901,0.859862,0.924358,1.075217,0.952211,...,1.032950,1.211550,0.465591,1.045382,1.119646,1.003923,0.717951,0.550652,0.764724,1.036992
4,4,0.454793,1.268686,0.000000,0.455023,0.455299,0.405792,0.857593,0.746529,1.216144,...,0.474285,0.354229,0.340511,0.864947,0.748424,0.411726,0.000000,0.290513,0.000000,0.060355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6894,7171,0.925742,0.212285,0.334050,1.029461,0.964702,0.806240,0.943946,0.757497,0.776653,...,0.480705,0.625817,0.514668,1.119148,0.750753,1.052949,0.168634,0.207517,0.000000,0.038046
6895,7172,1.115040,0.522502,0.521909,0.764038,1.113742,1.168937,1.004391,1.287956,0.973669,...,0.515929,1.023741,0.912149,1.207975,1.056919,1.672876,1.032393,1.155752,0.813821,0.206309
6896,7173,0.326512,0.486537,0.000000,0.487058,0.443303,0.986362,1.054962,0.252676,0.524198,...,0.000000,0.482575,0.327401,2.141854,0.895465,0.395234,0.559333,0.196024,0.309758,0.019076
6897,7174,0.725369,0.523844,0.941495,1.048326,0.855360,0.705701,0.850582,0.933961,0.785818,...,0.976515,0.883538,0.317697,0.958845,0.805494,0.885901,0.247095,0.609666,2.318234,0.438914


In [5]:
user_df = user_avg_feat.copy().merge(user_avg_category, on="user_id", how="left")
user_df

,user_id,avg_feat_0,avg_feat_1,avg_feat_2,avg_feat_3,avg_feat_4,avg_feat_5,avg_feat_6,avg_feat_7,avg_feat_8,...,avg_category_30,avg_category_31,avg_category_32,avg_category_33,avg_category_34,avg_category_35,avg_category_36,avg_category_37,avg_category_38,avg_category_39
0,0,1.279674,0.977637,0.909355,0.902536,0.762750,1.156619,0.857332,1.116271,1.157333,...,1.410021,1.178845,0.787215,1.219192,0.985790,1.023252,0.876753,0.836855,0.586343,1.098977
1,1,0.365590,0.747119,0.285845,0.183951,1.644048,0.932794,0.908008,1.039110,0.993866,...,1.234668,1.083152,0.605532,0.635479,0.706868,1.121345,0.992612,1.489968,3.278239,0.253628
2,2,0.376160,0.728525,0.481671,0.558891,0.910193,0.685176,0.513376,0.658626,0.668409,...,0.885171,0.539263,0.461030,0.678805,0.598511,0.581113,0.464605,0.446583,0.000000,0.000000
3,3,1.450146,0.950613,0.775522,0.796545,1.047035,1.208812,0.824149,1.009434,1.053580,...,1.032950,1.211550,0.465591,1.045382,1.119646,1.003923,0.717951,0.550652,0.764724,1.036992
4,4,0.000000,0.476605,1.010854,0.000000,0.455023,0.330529,0.491453,0.716446,0.744431,...,0.474285,0.354229,0.340511,0.864947,0.748424,0.411726,0.000000,0.290513,0.000000,0.060355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6894,7171,0.603449,0.899103,0.717087,0.526626,0.739019,0.910404,0.662487,0.884767,0.764045,...,0.480705,0.625817,0.514668,1.119148,0.750753,1.052949,0.168634,0.207517,0.000000,0.038046
6895,7172,0.819361,1.042957,0.793120,0.496253,0.647006,0.962898,0.987411,1.039611,1.269542,...,0.515929,1.023741,0.912149,1.207975,1.056919,1.672876,1.032393,1.155752,0.813821,0.206309
6896,7173,0.293560,0.273140,1.719037,0.000000,0.375374,0.478506,0.726166,0.684708,0.609157,...,0.000000,0.482575,0.327401,2.141854,0.895465,0.395234,0.559333,0.196024,0.309758,0.019076
6897,7174,1.071403,0.710077,0.527994,0.851187,0.932450,0.849032,0.673943,0.852725,0.865874,...,0.976515,0.883538,0.317697,0.958845,0.805494,0.885901,0.247095,0.609666,2.318234,0.438914


# CREATING VIDEO VECTORS

In [6]:
video_feat_df = item_categories_cleaned.copy()
video_feat_df = video_feat_df.explode(column="feat")
video_feat_df = pd.crosstab(video_feat_df["video_id"], video_feat_df["feat"])
video_feat_df.columns = [f'feat_{int(col)}' for col in video_feat_df.columns]
video_feat_df

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30
video_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10722,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10723,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10724,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
video_category_df = caption_category_cleaned.copy()
video_category_df = pd.crosstab(video_category_df["video_id"], video_category_df["first_level_category_id"])
video_category_df.columns = [f'category_{int(col)}' for col in video_category_df.columns]
video_category_df

,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_10,...,category_30,category_31,category_32,category_33,category_34,category_35,category_36,category_37,category_38,category_39
video_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10722,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10723,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
10724,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
cols_to_drop = ["valid_play_cnt", "like_cnt", "comment_cnt", "share_cnt", "follow_cnt", "collect_cnt", "download_cnt"]
video_df = item_daily_features_cleaned.copy()
video_df["score"] = video_df["valid_play_cnt"] * 0.1 + video_df["like_cnt"] * 0.2 + video_df["comment_cnt"] * 0.3 + video_df["share_cnt"] * 0.5 + video_df["collect_cnt"] * 0.5 + video_df["download_cnt"] * 0.5 + video_df["follow_cnt"] * 1
video_df = video_df.drop(columns=cols_to_drop)
video_df["score"] = np.log10(video_df["score"])
video_df["score"] = video_df["score"].replace(-np.inf, np.nan)



full_date_range = pd.date_range(start='2020-06-23', end='2020-09-10')
video_ids = video_df['video_id'].unique()
full_index = pd.MultiIndex.from_product([video_ids, full_date_range], names=['video_id', 'date'])

video_df = (
    video_df
    .set_index(['video_id', 'date'])
    .reindex(full_index)
    .reset_index()
)

video_df['upload_dt'] = (
    video_df
    .groupby('video_id')['upload_dt']
    .transform(lambda x: x.ffill().bfill())
)

video_df['score_masked'] = video_df.apply(
    lambda row: row['score'] if row['date'] >= row['upload_dt'] else np.nan,
    axis=1
)

video_df['score'] = video_df['score'].fillna(0)

video_df = video_df.sort_values(['video_id', 'date'])

def rolling_trend_masked(group):
    group = group.sort_values('date').copy()
    upload_dt = group['upload_dt'].iloc[0]

    # Set index to date for rolling
    group = group.set_index('date')
    valid = group.index > upload_dt  # Exclude upload day
    trend = pd.Series(np.nan, index=group.index)

    trend[valid] = (
        group['score_masked'][valid]
        .shift(1)
        .rolling('7D', min_periods=1)
        .mean()
    )

    group['trend_score'] = trend
    group = group.reset_index()
    return group[['video_id', 'date', 'trend_score']]

trend_scores = (
    video_df
    .groupby('video_id', group_keys=False)
    .apply(rolling_trend_masked)
    .reset_index(drop=True)
    .fillna(0)
)


video_df = pd.merge(video_df, trend_scores, on=['video_id', 'date'], how='left')
video_df['video_duration'] = (
    video_df
    .groupby('video_id')['video_duration']
    .transform(lambda x: x.ffill().bfill())
)
video_df = video_df.drop(columns=["upload_dt", "score", "score_masked"])
video_df

/home/sah/miniforge3/envs/recommender/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/tmp/ipykernel_59892/2724622292.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(rolling_trend_masked)


,video_id,date,video_duration,trend_score
0,0,2020-06-23,5966.0,0.0
1,0,2020-06-24,5966.0,0.0
2,0,2020-06-25,5966.0,0.0
3,0,2020-06-26,5966.0,0.0
4,0,2020-06-27,5966.0,0.0
...,...,...,...,...
700315,10727,2020-09-06,5666.0,0.0
700316,10727,2020-09-07,5666.0,0.0
700317,10727,2020-09-08,5666.0,0.0
700318,10727,2020-09-09,5666.0,0.0


In [42]:
video_df = video_df.merge(video_feat_df, on="video_id", how="left")
video_df = video_df.merge(video_category_df, on="video_id", how="left")
video_df

,video_id,date,video_duration,trend_score,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,...,category_30,category_31,category_32,category_33,category_34,category_35,category_36,category_37,category_38,category_39
0,0,2020-06-23,5966.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2020-06-24,5966.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2020-06-25,5966.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2020-06-26,5966.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2020-06-27,5966.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700315,10727,2020-09-06,5666.0,0.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
700316,10727,2020-09-07,5666.0,0.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
700317,10727,2020-09-08,5666.0,0.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
700318,10727,2020-09-09,5666.0,0.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [44]:
export_dir = "./exports"
if not os.path.exists(export_dir):
    os.makedirs(export_dir)
user_df.to_parquet(export_dir + "/user_df.pq")
video_df.to_parquet(export_dir + "/video_df.pq")